### Capstone Project
This notebook is mainly used for the capstone project of the 'Applied Data Science' course.

installations

In [31]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy

In [32]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [33]:
df_crime = pd.read_csv('MCI_2014_to_2018.csv')
print("Crime dataset contains %d unique neighbourhoods." % len(df_crime['Neighbourhood'].unique()))
# df_crime.head()

Crime dataset contains 141 unique neighbourhoods.


In [34]:
df_crime.groupby('Neighbourhood').count().head()

,X,Y,Index_,event_unique_id,occurrencedate,reporteddate,premisetype,ucr_code,ucr_ext,offence,...,occurrenceday,occurrencedayofyear,occurrencedayofweek,occurrencehour,MCI,Division,Hood_ID,Lat,Long,ObjectId
Neighbourhood,,,,,,,,,,,,,,,,,,,,,
Agincourt North (129),1008,1008,1008,1008,1008,1008,1008,1008,1008,1008,...,1008,1008,1008,1008,1008,1008,1008,1008,1008,1008
Agincourt South-Malvern West (128),1331,1331,1331,1331,1331,1331,1331,1331,1331,1331,...,1330,1330,1330,1331,1331,1331,1331,1331,1331,1331
Alderwood (20),460,460,460,460,460,460,460,460,460,460,...,460,460,460,460,460,460,460,460,460,460
Annex (95),2908,2908,2908,2908,2908,2908,2908,2908,2908,2908,...,2908,2908,2908,2908,2908,2908,2908,2908,2908,2908
Banbury-Don Mills (42),938,938,938,938,938,938,938,938,938,938,...,938,938,938,938,938,938,938,938,938,938


In [35]:
df_venues = pd.read_csv('toronto_venues.csv')
df_venues.head()

,Unnamed: 0,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa
